## SGF棋盘数据预处理

In [ ]:
# 环境配置
%cd /playground/sgd_deep_learning/sgd_rl/go
import sys
sys.path.append('./python')

In [ ]:
from dlgo.data import  GoDataProcessor # 单线程
from dlgo.data import  GoDataProcessorFast # 多线程

### 单线程处理
    * 每次执行，都会重新采样一遍，比较费时费力。 而且不同的encoder需要重新执行一遍。最好把数据按照encoder，新建一个目录然后归类。

    * 才处理了100局游戏数据，就已经很慢了25s。需要使用并发提速
        总共17 0000 局游戏，总处理代价11小时。这还是最简单的oneplane编码格式

    * todo 改造成pytorch data set 和 data loader的格式，方便一键训练模型

In [ ]:
go_data_process = GoDataProcessor(encoder='oneplane', data_directory='data')
train_data = go_data_process.load_go_data(data_type='train', num_samples=100)

### 多线程并发
        # 默认进程池数量=cpu cores num (6核心12线程)
        speedup 25s -> 6s 也就快了3-4倍，还能更快吗？

In [ ]:
go_data_process = GoDataProcessorFast(encoder='oneplane', data_directory='data')
train_data = go_data_process.load_go_data(data_type='train', num_samples=100)

### 线程池数调优
    cpu：3600X(6-core,12-process)
    100局游戏6s完成处理。12线程也就比单线程快了4倍左右，全量数据可以由11h缩短为3h

    采样100games
    线程池数:9, 100局游戏cost8.037
    线程池数:10, 100局游戏cost7.213
    线程池数:11, 100局游戏cost7.136
    线程池数:12, 100局游戏cost6.726  !较快
    线程池数:13, 100局游戏cost6.856
    线程池数:14, 100局游戏cost6.222  !最快
    线程池数:15, 100局游戏cost6.568
    线程池数:16, 100局游戏cost6.733
    线程池数:17, 100局游戏cost6.857
    线程池数:18, 100局游戏cost6.901
    线程池数:19, 100局游戏cost6.614

    采样200games
    线程池数:10, 100局游戏cost8.029
    线程池数:11, 100局游戏cost7.849
    线程池数:12, 100局游戏cost7.311  !较快
    线程池数:13, 100局游戏cost7.666
    线程池数:14, 100局游戏cost7.521
    线程池数:15, 100局游戏cost7.199  !最快

    结论：选取默认的虚拟核数量就好
    

In [ ]:
import time
process_num=12 # 默认是6核心12线程
def test_time(process_num):
    t1 = time.time()
    go_data_process = GoDataProcessorFast(encoder='oneplane', data_directory='data', process_num=process_num)
    train_data = go_data_process.load_go_data(data_type='train', num_samples=200)
    t2 = time.time()
    
    return "线程池数:{}, 100局游戏cost{:.3f}".format(process_num, t2-t1)

# 实验使用，默认不执行
# logs = []
# for i in range(10, 16):
#     logs.append(test_time(process_num=i))
    
# for s in logs:
#     print(s)